# 99% accurary on Traffic Sign Recognition using Fastai


## Objective

Use fastai and a pretrained model (resnet34) to classify traffic sign from the GTSRB dataset.
With a bit more code such a model can be used for automatic sign detection in a video feed.

SOME TEXT TO WRITE HERE.

<br>
<strong style="color:red;">Please Upvote my kernel and keep it in your favourite section if you think it is helpful.</strong>


<div style="color:white;background-color:#337ab7;padding:15px;width:100%;margin-top:50px;">Table of contents</div>


* [**Introduction**](#introduction)
  1. [What is Fastai?](#fastai)
  1. [About the dataset](#gtsrb-dataset)


* [**Library**](#library)
  1. [Fastai installation](#installation)
  1. [Import Libraries](#librairies)


* [**Load and inspect your data**](#load-inspect-data)
  1. [Convert numerical labels to human readable labels](#convert-labels)
  1. [Load training data](#training-data)
  1. [Inspect target class distribution](#dataset-distribution)
  1. [Visualize target class](#target-class)
  1. [Visualize real images for each class](#visualize-images-per-class)


* [**Train a learner (model)**](#train-learner)
  1. [Train the model](#train)
  1. [Save the trained model](#save-model)
  1. [Interpret the results](#interpretation)
  1. [The most confused](#confusion-matrix)
  1. [Can a human do better on wrong predictions?](#can-human-do-better)
  1. [Predict using trained model](#test-prediction)


* [**Predict unseen data**](#predict-unseen-data)
  1. [Load test data](#test-data)
  1. [Get predictions for test data](#predict-test-data)
  1. [Test data accuracy](#test-data-accuracy)
  1. [Is 99% accuracy enough for real world usage?](#real-world-usage)

# Introduction <a name="introduction"></a>

### What is Fastai? <a name="fastai"></a>


> fastai is a deep learning library which provides practitioners with high-level components that can quickly and easily provide state-of-the-art results in standard deep learning domains, and provides researchers with low-level components that can be mixed and matched to build new approaches. It aims to do both things without substantial compromises in ease of use, flexibility, or performance.
>
> -- <cite>[About Fastai](https://docs.fast.ai/#About-fastai)</cite>

Basically it's a library that allows both beginner and advanced deep learning practitioners to produce the best result possible with very few lines of codes.


### About the dataset <a name="gtsrb-dataset"></a>

The German Traffic Sign Benchmark (GTSRB) is a multi-class single-image classification dataset that was use during a challenge held at the International Joint Conference on Neural Networks (IJCNN) in 2011.  
The dataset has 43 classes and a little more than 50,000 images.

Please note that we are using the updated version of the dataset.  
The 2011 version of the data has some flaws which are fixed in the final version.

The original dataset is on the [INI Benchmark Website](http://benchmark.ini.rub.de/)


# Library <a name="library"></a>

### Fastai installation <a name="installation"></a>

Since fastai is already installed in kaggle notebooks, you don't need to install it again.  
However, if it's missing in your notebook, you can run the below command to intall it.

```
!pip install fastai
```

### Import Libraries <a name="librairies"></a>

In normal python code `import *` is not recommended. But in fastai things are so optimized that it allows you to import everything you need to tackle your problem without slowing things down.

In [ ]:
from fastai.vision.all import *

import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns

from sklearn.metrics import accuracy_score

# Load and inspect your data <a name="load-inspect-data"></a>

### Convert numerical labels to human readable labels <a name="convert-labels"></a>

Due to the fact that the categories (classes) are numbers, it can be quite hard to relate to them. Let's convert them.

But before we will load the dataframe holding our data.

In [ ]:
train_df = pd.read_csv('../input/gtsrb-german-traffic-sign/Train.csv')
# display a sneak peek of the data
train_df.head()

In [ ]:
print(f'Number of classes: {train_df.ClassId.unique().shape[0]}')

The column `ClassId` holds the target class for each sample. As you see it's an integer column. Thanks to [Mykola's notebook](https://www.kaggle.com/meowmeowmeowmeowmeow/road-signs-recognition) we can convert this to readable labels to make it easier to interpret the results of our model.

In [ ]:
labels = ['20 km/h', '30 km/h', '50 km/h', '60 km/h', '70 km/h', '80 km/h', '80 km/h end', '100 km/h', '120 km/h', 'No overtaking',
               'No overtaking for trucks', 'Crossroad with secondary way', 'Main road', 'Give way', 'Stop', 'Road up', 'Road up for truck', 'No entry',
               'Other dangerous', 'Turn left', 'Turn right', 'Winding road', 'Hollow road', 'Slippery road', 'Narrowing road', 'Roadwork', 'Traffic light',
               'Pedestrian', 'Children', 'Bike', 'Snow', 'Deer', 'End of the limits', 'Only right', 'Only left', 'Only straight', 'Only straight and right', 
               'Only straight and left', 'Take right', 'Take left', 'Circle crossroad', 'End of overtaking limit', 'End of overtaking limit for truck']
# add column with readable labels
train_df['Label'] = train_df['ClassId'].replace(sorted(train_df['ClassId'].unique()), labels)
# print updated df
train_df.head()

### Load training data <a name="training-data"><a/>

We need to use part of our training for validation purposes. Fastai already handles that by keeping aside 20% of the data as validation dataset. 
Since the images are not the same size we'll transform them to have the size 224x224 using `item_tfms`.

In [ ]:
dls = ImageDataLoaders.from_df(train_df, fn_col='Path', label_col='Label', path='../input/gtsrb-german-traffic-sign/', seed=42, item_tfms=Resize(224))

Let's inspect our training data.

In [ ]:
dls.train.show_batch(max_n=7, nrows=1)

We also need to inspect the validation set.

**N.B.**: One thing that is noticeable is that some images in the data are quite dark. My guess is that the model might struggle with these one. In the real world the lights from the car can help make the signs more visible at the night.

In [ ]:
dls.valid.show_batch(max_n=7, nrows=1)

### Inspect target class distribution <a name="dataset-distribution"></a>

This is simply a way to give us an idea of how many samples each class has in the training data. This helps answer the question: how much of each class our model will learn from?

In [ ]:
fig, ax = plt.subplots(figsize=(25, 6))
ax.set_title('Training classes distribution')
ax.set_xlabel('Class')
ax.set_ylabel('Count')

chart = sns.countplot(train_df.Label, ax=ax, orient="v")
ax.set_xlabel('Labels');
ax.set_xticklabels(chart.get_xticklabels(), rotation=45, horizontalalignment='right');

As you can see, some classes are less represented than others. If many of these classes are part of the classes the model has a hard time detecting, it might means that we need more or better samples for each in order to improve our results. 

In [ ]:
label_counts = train_df.Label.value_counts()
less_represented = label_counts[label_counts < 300]
less_represented

### Visualize target class <a name="target-class"></a>

To better understand the target class it might help to see what the traffic sign should look like.

In [ ]:
meta_df = pd.read_csv('../input/gtsrb-german-traffic-sign/Meta.csv')

In [ ]:
def display_images_with_labels(images, labels, ncols=7):
    plt.figure(figsize=(25,12))
    plt.subplots_adjust(hspace=0.5)
    nrows = len(images) / ncols + 1
    for i, image in enumerate(images):
        img_idx = i + 1
        ax = plt.subplot(nrows, ncols, img_idx, title=labels[i], frame_on=False)
        ax.imshow(image)
        ax.axis("off")

# build the list of images and display them
meta_df['ImgPath'] = "../input/gtsrb-german-traffic-sign/" + meta_df["Path"]
images = meta_df['ImgPath'].apply(mpimg.imread)
img_labels = meta_df['ClassId'].replace(sorted(meta_df['ClassId'].unique()), labels)
display_images_with_labels(images, img_labels)

### Visualize real images for each class <a name="visualize-images-per-class"></a>

Now that we have a visual representation of each class, less see how each of them look like in our dataset.

In [ ]:
# build the list of images to show
train_df['ImgPath'] = "../input/gtsrb-german-traffic-sign/" + train_df["Path"]
uniq_train_df = train_df.drop_duplicates(subset=['Label'])
images = uniq_train_df['ImgPath'].apply(mpimg.imread)
img_labels = uniq_train_df['Label'].values

display_images_with_labels(images, img_labels)

# Train a learner (model) <a name="train-learner"><a/>


### Train the model <a name="train"></a>

We'll use **error_rate** and **accuracy** as metrics for our model. The error rate will help us measure the performance of our model and interpret its results. Whereas the accuracy will help us understand how good is our model at choosing the right answer.

In [ ]:
learn = cnn_learner(dls, resnet34, metrics=[error_rate, accuracy], model_dir=Path("/kaggle/working/model"))
learn.fine_tune(8)

### Save the trained model <a name="save-model"></a>

In [ ]:
learn.export('/kaggle/working/export.pkl')

### Interpret the results <a name="interpretation"></a>

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

`plot_top_losses` lets us visualize the samples with the higher loss during training. Meaning that we get to see on which images the performance of the model was worst. Keep in mind that a higher loss doesn't equate to a wrong prediction.

As you can see below, the higher the loss, the lower quality of the image.

Some of the images were enlarged to ensure size uniformity. Others are probably not centered in the **224x224** square that was cropped from the original image.

In [ ]:
interp.plot_top_losses(15, nrows=3)
plt.tight_layout()

### The most confused <a name="confusion-matrix"></a>

In most cases a confusion matrix helps understand why some classes are being confused with others. But in this case, there is just too many classes to make sense out of it.

In [ ]:
interp.plot_confusion_matrix(figsize=(25,12))

Luckily, **fastai** provides a helper function that will allow us to take a look at the categories that are the most confused.

The returned list is presented as: **actual / predicted / # of occurences**.

In [ ]:
mc = interp.most_confused()
mc

The classes that are the most confused are among the classes where the model had the worst performances.  
We can possibly use data augmentation technique to improve the results, provide better images or make sure we grab the right portion of the image for our training.

Contrary to what we thought at the beginning, none of the `less_represented` classes are part of the `most_confused` thanks to transfert learning. With transfert learning we can have very few number of sample and still achieve great results.  
This also means that it's not only about the quantity of the samples but quality of it.


### Can a human do better on wrong predictions? <a name="can-human-do-better"></a>

Given this dataset, a short answer is **No**.

For many of the images where the model had it wrong, it will be difficult for the naked eye to recognize the sign due to the darkness of the image.


### Predict using trained model <a name="test-prediction"></a>

In [ ]:
# load the model
learn_inf = load_learner('/kaggle/working/export.pkl')

In [ ]:
# helper func to predict and provide probability.
def predict(img_path, proba=False):
    pred,pred_idx,probs = learn_inf.predict(img_path)
    return [pred,pred_idx,probs] if proba else pred

# test predict function
pred,pred_idx,probs = predict('../input/gtsrb-german-traffic-sign/Test/00000.png', proba=True)

# Test prediction
print(f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}')

# Predict unseen data <a name="predict-unseen-data"></a>

### Load test data <a name="test-data"></a>

In [ ]:
# load test data
test_df = pd.read_csv('../input/gtsrb-german-traffic-sign/Test.csv')

In [ ]:
# add full path for each image
test_df['ImgPath'] = '../input/gtsrb-german-traffic-sign/' + test_df.Path

### Get predictions for test data <a name="predict-test-data"></a>

The `%%capture` magic method captures the output printed by `apply`. In this case, it's an empty new line after each prediction. We don't need that.

In [ ]:
%%capture
# Add prediction for each image in the test set
test_df['Category'] = test_df.ImgPath.apply(predict)

### Test data accuracy <a name="test-data-accuracy"></a>

In [ ]:
# convert numerical category to labels
test_df['Label'] = test_df['ClassId'].replace(sorted(test_df['ClassId'].unique()), labels)

In [ ]:
acc = accuracy_score(test_df['Label'], test_df['Category'])
# acc = 0.9884402216943785
print(f"Test data accuracy = {acc:.2f}")

### Is 99% accuracy enough for real world usage? <a name="real-world-usage"></a>

Yes and no. Yes if we have an optimal environment. This means the sign needs to be well lit and not obstructed (by an object, heavy rain or snowstorm).

Although the model has a nearly perfect score with very poor images, we might need further testing. Possibly hooking it to a camera mounted in a car and see how it performs in different scenarios such as:
  * **Night time**
  * **Heavy rain**
  * **Snow storm**
  * **Obstructed by an object**
  * **The car moving at high speed**

We might also need to test at which distance does the sign needs to be from a moving car for the model to give an accurate prediction. This might help set a safety threshold where the prediction is not to be trusted unless the distance is below that threshold.